In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Covers"

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime as dt
import datetime
from datetime import timedelta as td
import collections
import re
import os
import numpy as np
#pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth', 220)
pd.options.mode.chained_assignment = None

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
pd.set_option('display.max_rows', None) 
path="/content/drive/My Drive/Results/"
path2="/content/drive/My Drive/"
www='https://contests.covers.com'
www2='https://www.covers.com'

conf=['America East','American Athletic Conference','Atlantic 10','Atlantic Coast','Atlantic Sun','Big 12','Big East','Big Sky','Big South','Big Ten','Big West','West Coast',
      'Colonial Athletic Association','Conference USA','Horizon League','Ivy League','Metro Atlantic Athletic','Mid-American','Mid-Eastern Athletic','Missouri Valley','Conf.','Tournament',
      'Mountain West','Northeast',' Ohio Valley','Pacific-12','Patriot League','Southeastern','Southern','Southland','Southwestern Athletic','Summit','Sun Belt','Western Athletic']

In [ ]:
d=dt.now()
d-=td(days=1)
date=d.strftime('%Y-%m-%d')
dat=d.strftime('%y-%m-%d')

#date='2020-07-24'
nfl_date='2020-09-24'
ncaaf_date='2020-09-24'
#dat='20-07-24'
all_sports={'MLB':'https://www.covers.com/Sports/MLB/Matchups?selectedDate={}'.format(date),
        'NBA':'https://www.covers.com/Sports/NBA/Matchups?selectedDate={}'.format(date),
        'NHL':'https://www.covers.com/Sports/NHL/Matchups?selectedDate={}'.format(date),
        'NCAAB':'https://www.covers.com/Sports/NCAAB/Matchups?selectedDate=2019-{}'.format(date),
        'NFL':'https://www.covers.com/Sports/NFL/Matchups?selectedDate={}'.format(nfl_date),
        'NCAAF':'https://www.covers.com/Sports/NCAAF/Matchups?selectedDate={}'.format(ncaaf_date)
        }

sports={}
for liga in ['NHL','MLB','NBA','NFL','NCAAB','NCAAF']:
  st=os.stat(path2 + f'{liga}.txt')
  sec_date=list(st)[8]-86400
  file_date=datetime.datetime.utcfromtimestamp(sec_date)
  if file_date.strftime('%Y-%m-%d') == date:
    sports[liga]=all_sports[liga]

display(sports)
ligi=list(sports.keys())
for k,v in sports.items():
  r=requests.get(v,headers=headers)
  locals()['s_'+k]=BeautifulSoup(r.text, 'html.parser')
  #f=open("/content/drive/My Drive/Today {}.html".format(k),'w')
  #f.write(r.text)
  #f.close()

{'MLB': 'https://www.covers.com/Sports/MLB/Matchups?selectedDate=2020-10-06',
 'NBA': 'https://www.covers.com/Sports/NBA/Matchups?selectedDate=2020-10-06'}

In [ ]:
def removeConf(row):
  for con in conf:
    if con in row['Matchup']:
      row['Matchup']=row['Matchup'].replace(con,'').strip()
  return row

def MLB(line):
    tt=line[0::13]  # Лига
    tt=list(map(lambda x: x.replace('American League','').replace('National League','').replace('Interleague','').replace('Chi. Cubs','Chicago') ,tt))
    t2=line[1::13]
    t1=line[7::13]
    t2=list(map(lambda x: x.split(' ')[0] if ' ' in x else x,t2)) # Команда2
    t1=list(map(lambda x: x.split(' ')[1] if ' ' in x else x,t1))  # Команда1
    s2=line[4::13]  # Тотал/Исход
    s1=line[6::13] # Исход/Тотал
    return tt,t1,t2,s1,s2

def NBA(line):
    tt=line[0::19]
    t2=list(map(lambda x: x.split(' ')[0],line[1::19]))
    ll=list(map(lambda x: x.split(' '),line[13::19]))
    t1=list(map(lambda x: x[1] if len(x)==2 else x[0],ll))
    s2=line[4::19]
    s1=line[16::19]
    return tt,t1,t2,s1,s2

def NHL(line):
    tt=line[0::16]  # Лига
    t2=list(map(lambda x: x.split(' ')[0],line[1::16])) # Команда2
    t1=list(map(lambda x: x.split(' ')[1],line[10::16]))  # Команда1
    s2=line[4::16]  # Тотал/Исход
    s1=line[13::16] # Исход/Тотал
    return tt,t1,t2,s1,s2

def NFL(line,w_day):
    tt=line[0::19]
    tt=list(map(lambda x: x.split(',')[0],tt))
    #tt=list(map(lambda x: x[:x.rfind(' ')],tt))
    tt=list(map(lambda x: x.replace(w_day,''),tt))
    t2=list(map(lambda x: x.split(' ')[0],line[1::19]))
    ll=list(map(lambda x: x.split(' '),line[13::19]))
    t1=list(map(lambda x: x[1] if len(x)==2 else x[0],ll))
    s2=line[4::19]
    s1=line[16::19]
    return tt,t1,t2,s1,s2

sports=pd.DataFrame(columns=['Date','League','Matchup','Teams','Total','Winner'])
pat=re.compile(' \(\d+\)')
for l in ligi:
  print(l)
  line=[]
  markers=[]
  datas={}
  sport=pd.DataFrame(columns=['Matchup','Teams','Total','Winner'])
  txt=open(path2+"{}.txt".format(l))
  for i,st in enumerate(txt):
    st=st.strip().replace('\n','').replace('\t','').replace(' (Neutral)','').replace(' NEUTRAL','').replace(' vs ',' at ').replace('Qualifying Round','')
    st=st.replace('Conference Quarterfinals','').replace('Conference Finals','').replace('Stanley Cup Finals','').replace('NBA Championship','')
    st=st.replace('Conference Semifinals','').replace('Wild Card Showdown','').replace('Divisional Playoff','')
    st=st.replace('Seeding Round Robin','')
    sk=st.find('(')
    if sk==0:
      if st[2]==')':
        st=st[4:]
      else:
        st=st[5:]
    x=re.search(pat,st)
    if x:
      rem=st[x.start():x.end()]
      st=st.replace(rem,'')
    line.append(st)
    if st.find('BET GRAPH')>-1 and l=='NCAAB':
        markers.append(i+1)
  txt.close()
  # Если берем NCAAB, то выкидываем строки там, где есть Off
  if l=='NCAAB':
    temp=[]
    for p in markers:
      temp+=line[p-19:p]
    line=temp

#MLB
  if l=='MLB':
    tt,t1,t2,s1,s2=MLB(line)
#NBA
  if l=='NBA':
    tt,t1,t2,s1,s2=NBA(line)
#NBA
  if l=='NHL':
    tt,t1,t2,s1,s2=NHL(line)
#NFL, NCAAF, NCAAB
  if l=='NFL' or l=='NCAAF' or l=='NCAAB':
    w_day=dt.strftime(d,'%A')
    tt,t1,t2,s1,s2=NFL(line,w_day)

  s2=list(map(lambda x: x.split(':')[0],s2))
  s1=list(map(lambda x: x.split(':')[0],s1))
  s2=list(map(lambda x: x.replace('PUSH','P'),s2))
  s1=list(map(lambda x: x.replace('PUSH','P'),s1))

  teams=[a+' at. '+b for a,b in zip(t2,t1)]
  sport['Teams']=teams
  sport['Matchup']=tt
  list_len=[x for x in range(len(tt))]
  total=list(map(lambda x: s1[x] if s1[x] in ['O','U','P'] else s2[x],list_len))
  sport['Total']=total
  winner=list(map(lambda x: t2[x] if s1[x] in ['O','U','P'] else t1[x],list_len))
  sport['Winner']=winner
  sport.insert(0,'League',l)
  sport.insert(0,'Date',dat)
  #display(sport)
  sports=sports.append(sport)
display(sports)
save=input('Записать?')
if save in ['y','н']:
  sports.to_csv(path+dat+'.csv',index=False)
  print('ЗАПИСАНО')

MLB
NBA


,Date,League,Matchup,Teams,Total,Winner
0,20-10-06,MLB,Miami at Atlanta,MIA at. ATL,O,ATL
1,20-10-06,MLB,Houston at Oakland,HOU at. OAK,U,HOU
2,20-10-06,MLB,NY Yankees at Tampa Bay,NYY at. TB,O,TB
3,20-10-06,MLB,San Diego at LA Dodgers,SD at. LAD,U,LAD
0,20-10-06,NBA,L.A. Lakers at Miami,LAL at. MIA,U,MIA


Записать?н
ЗАПИСАНО


In [ ]:
#ЗАМЕНА ДАТЫ
def removeConf(row):
  for con in conf:
    if con in row['Matchup']:
      row['Matchup']=row['Matchup'].replace(con,'')
  return row

#files=os.listdir(path)
#files=sorted(files)
for f in files[47:50]:
  ff=pd.read_csv(path+f)
  ff=ff.apply(removeConf,axis=1)
  #date=f[:-4]
  #ff.drop(labels=['Date'],inplace=True,axis=1)
  #ff.insert(0,'Date',date)
  display(ff)
  ff.to_csv(path+f,index=False)

,Date,League,Matchup,Teams,Total,Winner
0,20-01-09,NBA,Cleveland at Detroit,CLE at. DET,O,CLE
1,20-01-09,NBA,Boston at Philadelphia,BOS at. PHI,U,PHI
2,20-01-09,NBA,Portland at Minnesota,POR at. MIN,U,MIN
3,20-01-09,NBA,Houston at Oklahoma City,HOU at. OKC,U,OKC
4,20-01-09,NHL,Winnipeg at Boston,WIN at. BOS,O,BOS
5,20-01-09,NHL,Edmonton at Montreal,EDM at. MON,U,EDM
6,20-01-09,NHL,Arizona at Tampa Bay,ARI at. TB,U,TB
7,20-01-09,NHL,Vancouver at Florida,VAN at. FLA,O,FLA
8,20-01-09,NHL,New Jersey at NY Rangers,NJ at. NYR,O,NYR
9,20-01-09,NHL,Buffalo at St. Louis,BUF at. STL,O,STL
